## GitHub Organization

`project.name`
* `README.txt`
* `data_raw/` (raw data)
* `data/` (clean data)
* `notebook.ipynb`
* `src/` (helper functions) (optional)


## Setup

In [275]:
import os
import sys

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")

    # Clone repository if in Colab
    if not os.path.exists('/content/Dani_Math120_Final_Project/'):
        !git clone https://github.com/DaniCor3/Dani_Math120_Final_Project.git

    # Change to project directory
    os.chdir('/content/Dani_Math120_Final_Project/')

except ImportError:
    IN_COLAB = False
    print("Running locally")

# Add src directory to Python path
if 'src' not in sys.path:
    sys.path.append('src')

print(f"Current working directory: {os.getcwd()}")

Running in Google Colab
Current working directory: /content/Dani_Math120_Final_Project


In [276]:
import re # needed to clean up the words
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

PATH="/content/Dani_Math120_Final_Project/data_raw/Dune.txt"
with open(PATH, 'r') as f:
  text = f.read()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Cleaning the Data

In [277]:
separated={}
i=0

for s in re.split('= = = = = =', text):
  separated[i]=s
  i+=1

print(separated)

{0: 'Dune\nFrank Herbert\nCopyright 1965\nBook 1\nDUNE\n', 1: '\nA beginning is the time for taking the most delicate care that the balances are\ncorrect. This every sister of the Bene Gesserit knows. To begin your study of\nthe life of Muad\'Dib, then, take care that you first place him in his time: born\nin the 57th year of the Padishah Emperor, Shaddam IV. And take the most special\ncare that you locate Muad\'Dib in his place: the planet Arrakis. Do not be\ndeceived by the fact that he was born on Caladan and lived his first fifteen\nyears there. Arrakis, the planet known as Dune, is forever his place.\n-from "Manual of Muad\'Dib" by the Princess Irulan\nIn the week before their departure to Arrakis, when all the final scurrying\nabout had reached a nearly unbearable frenzy, an old crone came to visit the\nmother of the boy, Paul.\nIt was a warm night at Castle Caladan, and the ancient pile of stone that\nhad served the Atreides family as home for twenty-six generations bore that\nc

In [278]:
def makeCleanTokens(text):
  '''
  Makes a list containing every word in a text, excluding stop words
  '''
  if type(text)!=str:
    raise TypeError('text must be a string!')
  if text=='':
    raise ValueError('text cannot be empty!')

  text=text.lower() # so words that are the same but with different capitalization won't be counted differently
  tokens=[]
  for word in re.findall(r"\b[a-z|0-9]+",text):
    # looking for occurrences of letters and numbers
    if word in stop_words or word=='said':
      # removing stop words from the pool
      # since Frank Herbert rarely uses verbs other than "said" in dialogue, I will remove that from the pool too
      continue
    tokens.append(word)
    # adding to a list of the words in the text
  return tokens

In [279]:
# chapters will skip 0, 23, 39, 51, 52, 53, 54, 55
j=1
chDict={}

for i, s in separated.items():
  if i in [0,23,39,51,52,53,54,55]: # skipping title, sections, and appendices
    continue
  chDict[j]=s
  j+=1

tokenized=[]

for text in chDict.values():
  tokenized.append(makeCleanTokens(text))

chapters = pd.DataFrame({
    'Chapter': list(range(1,49)),
    'Text': list(chDict.values()),
    'Tokens': tokenized
})

chapters

,Chapter,Text,Tokens
0,1,\nA beginning is the time for taking the most ...,"[beginning, time, taking, delicate, care, bala..."
1,2,\nTo attempt an understanding of Muad'Dib with...,"[attempt, understanding, muad, dib, without, u..."
2,3,"\nThus spoke St. Alia-of-the-Knife: ""The Rever...","[thus, spoke, st, alia, knife, reverend, mothe..."
3,4,\nYou have read that Muad'Dib had no playmates...,"[read, muad, dib, playmates, age, caladan, dan..."
4,5,"\nYUEH (yu'e), Wellington (weling-tun), Stdrd ...","[yueh, yu, e, wellington, weling, tun, stdrd, ..."
5,6,\nHow do we approach the study of Muad'Dib's f...,"[approach, study, muad, dib, father, man, surp..."
6,7,"\nWith the Lady Jessica and Arrakis, the Bene ...","[lady, jessica, arrakis, bene, gesserit, syste..."
7,8,"\n""Yueh! Yueh! Yueh!"" goes the refrain. ""A mil...","[yueh, yueh, yueh, goes, refrain, million, dea..."
8,9,\nMany have marked the speed with which Muad'D...,"[many, marked, speed, muad, dib, learned, nece..."
9,10,\nWhat had the Lady Jessica to sustain her in ...,"[lady, jessica, sustain, time, trial, think, c..."


##Counting Words

In [280]:
def countWords(tokens):
  '''
  Counts words in a list of tokens, returning a dict of the words and their counts
  '''
  if type(tokens)!=list:
    raise TypeError('tokens must be a list!')
  if tokens==[]:
    raise ValueError('tokens cannot be empty!')

  wordCounts={}

  for word in tokens:
    if word in wordCounts:
      wordCounts[word]+=1 # increments word in dict
    else:
      wordCounts[word]=1 # puts word in dict

  return wordCounts

In [281]:
wordCounts=countWords(tokenized[0])
print(wordCounts)

{'beginning': 1, 'time': 3, 'taking': 1, 'delicate': 1, 'care': 3, 'balances': 1, 'correct': 1, 'every': 3, 'sister': 1, 'bene': 13, 'gesserit': 13, 'knows': 2, 'begin': 1, 'study': 1, 'life': 1, 'muad': 3, 'dib': 3, 'take': 7, 'first': 4, 'place': 5, 'born': 2, '57th': 1, 'year': 1, 'padishah': 2, 'emperor': 3, 'shaddam': 1, 'iv': 1, 'special': 2, 'locate': 1, 'planet': 7, 'arrakis': 12, 'deceived': 1, 'fact': 2, 'caladan': 4, 'lived': 2, 'fifteen': 3, 'years': 3, 'known': 3, 'dune': 4, 'forever': 1, 'manual': 1, 'princess': 1, 'irulan': 1, 'week': 1, 'departure': 1, 'final': 1, 'scurrying': 1, 'reached': 1, 'nearly': 1, 'unbearable': 1, 'frenzy': 1, 'old': 32, 'crone': 1, 'came': 2, 'visit': 1, 'mother': 41, 'boy': 2, 'paul': 47, 'warm': 1, 'night': 1, 'castle': 3, 'ancient': 3, 'pile': 1, 'stone': 1, 'served': 1, 'atreides': 5, 'family': 2, 'home': 1, 'twenty': 1, 'six': 1, 'generations': 1, 'bore': 1, 'cooled': 1, 'sweat': 3, 'feeling': 3, 'acquired': 1, 'change': 2, 'weather': 1, 

In [282]:
def sortDict(d):
  '''
  Sorts a dict that has integer values
  '''
  if type(d)!= dict:
    raise TypeError('d must be a dict!')
  if d=={}:
    raise ValueError('d cannot be empty!')

  sortList=[]
  for word, n in d.items():
    if type(n)!=int:
      raise TypeError('d must only have integer values!')

    if len(sortList)==0:
      sortList.append(word)
      continue

    elif len(sortList)==1:
      if d[sortList[0]]>n:
        sortList.append(word)
        continue
      else:
        sortList.insert(0,word)
        continue

    for j in range(len(sortList)):
      if d[word]>d[sortList[j]]:
        sortList.insert(j,word)
        break

    if word not in sortList: # after everything, if word isn't in sorted, it is appended
      sortList.append(word)

  sorted={}
  for word in sortList:
    sorted[word]=d[word]

  return sorted

In [283]:
print(sortDict(wordCounts))

{'paul': 47, 'mother': 41, 'old': 32, 'hand': 32, 'woman': 27, 'jessica': 27, 'pain': 17, 'one': 16, 'gom': 15, 'jabbar': 15, 'felt': 15, 'fear': 15, 'bene': 13, 'gesserit': 13, 'voice': 13, 'human': 13, 'reverend': 13, 'arrakis': 12, 'door': 12, 'test': 12, 'box': 11, 'could': 10, 'like': 10, 'see': 9, 'eyes': 9, 'know': 9, 'many': 9, 'looked': 9, 'away': 9, 'duke': 8, 'must': 8, 'neck': 8, 'take': 7, 'planet': 7, 'side': 7, 'heard': 7, 'us': 7, 'mind': 7, 'son': 7, 'tried': 7, 'terrible': 7, 'room': 6, 'reverence': 6, 'little': 6, 'kwisatz': 6, 'haderach': 6, 'stared': 6, 'would': 6, 'animal': 6, 'face': 6, 'saw': 6, 'back': 6, 'withdraw': 6, 'burning': 6, 'place': 5, 'atreides': 5, 'asked': 5, 'yet': 5, 'people': 5, 'without': 5, 'remain': 5, 'close': 5, 'black': 5, 'turned': 5, 'truthsayer': 5, 'thought': 5, 'stopped': 5, 'took': 5, 'right': 5, 'look': 5, 'truth': 5, 'told': 5, 'drug': 5, 'first': 4, 'caladan': 4, 'dune': 4, 'bed': 4, 'light': 4, 'yes': 4, 'well': 4, 'within': 4, '

In [284]:
def chWordCounts(tokenized):
  '''
  Counts and sorts words from a list of lists of tokens in each chapter
  Returns two lists: one containing the sorted word counts for each chapter and the other containing the top 5 words for each chapter.
  '''
  if type(tokenized)!=list:
    raise TypeError('tokenized must be a list!')
  if tokenized==[]:
    raise ValueError('tokenized cannot be empty!')

  chCounter=[]
  for ch in tokenized:
    d = sortDict(countWords(ch))
    chCounter.append(d)

  return chCounter

In [285]:
def top5Chapter(chCounter):
  '''
  Separates the top 5 word entries per chapter.
  '''
  if type(chCounter)!=list:
    raise TypeError('chCounter must be a list!')
  if chCounter==[]:
    raise ValueError('chCounter cannot be empty!')

  chTop5Words=[]
  chTop5Nums=[]
  for ch in chCounter:
    words=[]
    nums=[]
    for i in range(5):
      words.append(list(ch.keys())[i])
      nums.append(list(ch.values())[i])
    chTop5Words.append(words)
    chTop5Nums.append(nums)

  return chTop5Words, chTop5Nums

In [286]:
chCounter=chWordCounts(tokenized)
print(chCounter)
top5Chapter(chCounter)

[{'paul': 47, 'mother': 41, 'old': 32, 'hand': 32, 'woman': 27, 'jessica': 27, 'pain': 17, 'one': 16, 'gom': 15, 'jabbar': 15, 'felt': 15, 'fear': 15, 'bene': 13, 'gesserit': 13, 'voice': 13, 'human': 13, 'reverend': 13, 'arrakis': 12, 'door': 12, 'test': 12, 'box': 11, 'could': 10, 'like': 10, 'see': 9, 'eyes': 9, 'know': 9, 'many': 9, 'looked': 9, 'away': 9, 'duke': 8, 'must': 8, 'neck': 8, 'take': 7, 'planet': 7, 'side': 7, 'heard': 7, 'us': 7, 'mind': 7, 'son': 7, 'tried': 7, 'terrible': 7, 'room': 6, 'reverence': 6, 'little': 6, 'kwisatz': 6, 'haderach': 6, 'stared': 6, 'would': 6, 'animal': 6, 'face': 6, 'saw': 6, 'back': 6, 'withdraw': 6, 'burning': 6, 'place': 5, 'atreides': 5, 'asked': 5, 'yet': 5, 'people': 5, 'without': 5, 'remain': 5, 'close': 5, 'black': 5, 'turned': 5, 'truthsayer': 5, 'thought': 5, 'stopped': 5, 'took': 5, 'right': 5, 'look': 5, 'truth': 5, 'told': 5, 'drug': 5, 'first': 4, 'caladan': 4, 'dune': 4, 'bed': 4, 'light': 4, 'yes': 4, 'well': 4, 'within': 4, 

([['paul', 'mother', 'old', 'hand', 'woman'],
  ['baron', 'piter', 'feyd', 'rautha', 'duke'],
  ['jessica', 'paul', 'mother', 'old', 'woman'],
  ['paul', 'halleck', 'hawat', 'thought', 'gurney'],
  ['paul', 'yueh', 'book', 'fremen', 'thought'],
  ['duke', 'paul', 'son', 'father', 'arrakis'],
  ['jessica', 'mapes', 'lady', 'thought', 'know'],
  ['jessica', 'yueh', 'people', 'like', 'wellington'],
  ['paul', 'room', 'could', 'would', 'seeker'],
  ['jessica', 'door', 'room', 'paul', 'hawat'],
  ['men', 'duke', 'room', 'gurney', 'halleck'],
  ['duke', 'hawat', 'paul', 'fremen', 'leto'],
  ['duke', 'hawat', 'lord', 'could', 'leto'],
  ['duke', 'paul', 'must', 'father', 'could'],
  ['kynes', 'duke', 'paul', 'halleck', 'man'],
  ['jessica', 'kynes', 'water', 'duke', 'paul'],
  ['jessica', 'duke', 'hawat', 'one', 'lady'],
  ['leto', 'yueh', 'duke', 'tooth', 'hand'],
  ['jessica', 'baron', 'piter', 'one', 'paul'],
  ['yueh', 'sardaukar', 'duke', 'thought', 'house'],
  ['baron', 'piter', 'duke',

In [287]:
def bookWordCounts(chCounter):
  '''
  Combines word counts per chapter into a total word count
  '''
  if type(chCounter)!=list:
    raise TypeError('chCounter must be a list!')
  if chCounter==[]:
    raise ValueError('chCounter cannot be empty!')

  totalCount={}
  for ch in chCounter:
    for word, count in ch.items():
      if word in totalCount.keys():
        totalCount[word]+=count
      else:
        totalCount[word]=count
  totalCount=sortDict(totalCount)

  return totalCount

In [288]:
def bookTop5(totalCount):
  '''
  Finds the top 5 words over all
  '''
  if type(totalCount)!=dict:
    raise TypeError('totalCount must be a dict!')
  if totalCount=={}:
    raise ValueError('totalCount cannot be empty!')

  totalTop5={}
  for word, count in totalCount.items():
    totalTop5[word]=count
    if len(totalTop5.keys())>=5:
      break

  return totalTop5

In [289]:
totalCount=bookWordCounts(chCounter)
print(totalCount)
bookTop5(totalCount)

{'paul': 1711, 'jessica': 895, 'one': 626, 'thought': 617, 'baron': 582, 'duke': 568, 'could': 527, 'man': 493, 'hawat': 429, 'fremen': 413, 'stilgar': 401, 'mother': 391, 'must': 378, 'asked': 373, 'back': 366, 'know': 361, 'see': 350, 'gurney': 348, 'eyes': 339, 'saw': 335, 'way': 333, 'like': 331, 'us': 330, 'kynes': 329, 'hand': 326, 'water': 322, 'voice': 315, 'time': 300, 'men': 285, 'sand': 279, 'would': 278, 'looked': 271, 'emperor': 267, 'around': 262, 'face': 259, 'chani': 259, 'felt': 255, 'old': 253, 'arrakis': 251, 'turned': 249, 'feyd': 228, 'father': 227, 'people': 226, 'across': 222, 'leto': 221, 'came': 214, 'halleck': 210, 'away': 206, 'thing': 206, 'place': 203, 'desert': 203, 'two': 201, 'room': 199, 'spice': 199, 'rautha': 198, 'think': 197, 'another': 193, 'left': 192, 'woman': 190, 'right': 189, 'sardaukar': 187, 'harkonnen': 184, 'even': 181, 'lord': 180, 'heard': 177, 'mind': 177, 'son': 176, 'muad': 173, 'dib': 173, 'door': 172, 'head': 172, 'without': 166, 't

{'paul': 1711, 'jessica': 895, 'one': 626, 'thought': 617, 'baron': 582}

In [290]:
oneTimeWords=[]
for word, count in totalCount.items():
  if count==1:
    oneTimeWords.append(word)

print(f'One-time words: {oneTimeWords}')
print(f'Number of one-time words: {len(oneTimeWords)}')

One-time words: ['balances', '57th', 'locate', 'deceived', 'visit', 'vaulted', 'bulky', 'spiderwebs', 'untuned', 'slyness', 'faculties', 'thump', 'mouthed', 'quasi', 'arouses', 'predictions', 'rigidly', 'caid', 'census', 'regate', 'aortal', 'enriched', 'flooding', 'overload', 'obtain', 'extend', 'victims', 'extinct', 'destroys', 'perceptual', 'bodily', 'strive', 'permanence', 'bustle', 'emotionless', 'semiformal', 'dressing', 'syubi', 'tapestried', 'farmlands', 'petulant', 'curtsy', 'pulsed', 'gestalten', 'maternal', 'ingredient', 'enjoin', 'politeness', 'solidly', 'cube', 'compulsions', 'prickling', 'gums', 'silvery', 'musky', 'aumas', 'manageable', 'denying', 'sirra', 'headsman', 'axe', 'trapper', 'flex', 'burns', 'gasps', 'agonized', 'crisping', 'charred', 'bathing', 'withstood', 'withdrawing', 'lightless', 'volition', 'stump', 'astonished', 'induction', 'maiming', 'uncurled', 'bitten', 'axis', 'hating', 'interrupts', 'enslave', 'butlerian', 'counterfeit', 'revolt', 'emphasizes', 'm

## Sentiment Analysis

In [291]:
chSentiments=[]

for ch in chDict.values():
  sent=sid.polarity_scores(ch)['compound']
  chSentiments.append(sent)

for n in range(len(chSentiments)):
  print(f'{n+1}\t{chSentiments[n]}')

1	-0.9985
2	0.9997
3	-0.9608
4	0.9848
5	0.9151
6	0.9928
7	0.9922
8	0.9399
9	-0.6572
10	0.9952
11	0.9718
12	0.9992
13	0.9791
14	-0.9961
15	-0.7074
16	0.9997
17	-0.9989
18	-0.9985
19	-0.9891
20	0.7187
21	-0.9995
22	-0.9998
23	-0.996
24	-0.9994
25	0.9296
26	-0.9953
27	-0.9542
28	0.9401
29	-0.9958
30	0.9414
31	0.9964
32	0.9983
33	-0.9975
34	-0.9955
35	0.988
36	-0.9894
37	-0.5225
38	0.9508
39	-0.9968
40	-0.9994
41	0.9988
42	0.9708
43	-0.9997
44	-0.9982
45	-0.9962
46	-0.9588
47	-0.999
48	-0.9998


## Data

In [292]:
chTop5Words, chTop5Nums=top5Chapter(chWordCounts(tokenized))

chDupes=[]
sentDupes=[]
ranks=[]
top5WordsCombined=[]
top5NumsCombined=[]
for i in range(48):
  for j in range(5):
    sentDupes.append(chSentiments[i])
    ranks.append(j+1)
    chDupes.append(i+1)
    top5WordsCombined.append(chTop5Words[i][j])
    top5NumsCombined.append(chTop5Nums[i][j])

duneData=pd.DataFrame({
    'Chapter': chDupes,
    'Sentiment': sentDupes,
    'Rank': ranks,
    'Word': top5WordsCombined,
    'Count': top5NumsCombined
})

duneData

,Chapter,Sentiment,Rank,Word,Count
0,1,-0.9985,1,paul,47
1,1,-0.9985,2,mother,41
2,1,-0.9985,3,old,32
3,1,-0.9985,4,hand,32
4,1,-0.9985,5,woman,27
...,...,...,...,...,...
235,48,-0.9998,1,paul,212
236,48,-0.9998,2,emperor,57
237,48,-0.9998,3,gurney,47
238,48,-0.9998,4,feyd,46


## Visualizations

In [293]:
figWords=px.bar(duneData, x='Chapter', y='Count', color='Word', title='Top 5 Words Per Chapter in Dune', hover_name='Word')
figWords.show()

In [294]:
figSent=go.Figure()
figSent.add_trace(go.Bar(x=list(range(1,49)),y=chSentiments))
figSent.update_layout(title='Sentiment Per Chapter of Dune',
                  xaxis_title='Chapter',
                  yaxis_title='Sentiment')
figSent.show()